Here we convert helpsteer2 to DPO format, we also only look at correctness and helpfulness as all other attributes has poor inter-rater agreement anyway.

- https://github.com/jondurbin/bagel/blob/3c7d2410a5a5ad2fd31b63529ef541135feefce4/bagel/data_sources/helpsteer.py#L4
- https://huggingface.co/datasets/sablo/HelpSteer_binarized
- https://huggingface.co/datasets/gx-ai-architect/HelpSteer2_DPO



actually helpsteer2 only has 2 options for each set, so it doesn't matter much?

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from datasets import load_dataset
import datasets

dataset = load_dataset('nvidia/HelpSteer2')
dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'response', 'helpfulness', 'correctness', 'coherence', 'complexity', 'verbosity'],
        num_rows: 20324
    })
    validation: Dataset({
        features: ['prompt', 'response', 'helpfulness', 'correctness', 'coherence', 'complexity', 'verbosity'],
        num_rows: 1038
    })
})

In [3]:
def scoring_func(row, weights=dict(helpfulness=1, correctness=2, coherence=0, complexity=0, verbosity=-0.5)):
    columns = sorted(weights.keys())
    w = np.array([weights[k] for k in columns])
    return (row[columns] * w).sum()

def transform2dpo(g, weights):
    g['score'] = g.apply(scoring_func, weights=weights, axis=1)
    g2 = g.sort_values('score')
    # assert g['score'].diff().max() > 0, 'no diff'
    chosen = g2.iloc[0]
    rejected = g2.iloc[-1]
    return dict(chosen=chosen.response, rejected=rejected.response, prompt=chosen.prompt, diff=g['score'].diff().max())

def transform_df(df, weights):
    df2 = df.groupby('prompt').apply(transform2dpo, weights, include_groups=True)
    df2 = pd.DataFrame.from_records(df2.values)
    return df2

In [4]:
def ds2dpo(dataset, weights):
    splits = list(dataset.keys())

    data = {}
    for split in splits:
        df = dataset[split].to_pandas()
        df2 = transform_df(df, weights)
        df2 = df2.query('diff > 0') # filter out the ones with no difference
        data[split] = datasets.Dataset.from_pandas(df2)
    dataset2 = datasets.DatasetDict(
        data
    ).select_columns(['prompt', 'chosen', 'rejected'])
    return dataset2

In [5]:
for key in ['helpfulness', 'correctness', 'coherence', 'complexity', 'verbosity']:
    weights = dict(
        helpfulness=0,
        correctness=0,
        coherence=0,
        complexity=0,
        verbosity=0,
    )
    weights[key] = 1
    dataset2 = ds2dpo(dataset, weights)
    dataset2.push_to_hub('helpsteer2_dpo_nonverbose', config_name=key)

/tmp/ipykernel_1010292/2093043050.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df2 = df.groupby('prompt').apply(transform2dpo, weights, include_groups=True)
/tmp/ipykernel_1010292/2093043050.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df2 = df.groupby('prompt').apply(transform2dpo, weights, include_groups=True)


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/2.39k [00:00<?, ?B/s]

In [6]:
# This is my preference. Correctness is the most important, followed by helpfulness, and verbosity punished. I ignore coherence, and complexity as they have low inter-rater agreement.
weights=dict(helpfulness=1, correctness=2, coherence=0, complexity=0, verbosity=-0.5)
dataset2 = ds2dpo(dataset, weights)
dataset2.push_to_hub('helpsteer2_dpo_nonverbose', config_name='composite')

/tmp/ipykernel_1010292/2093043050.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df2 = df.groupby('prompt').apply(transform2dpo, weights, include_groups=True)
/tmp/ipykernel_1010292/2093043050.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df2 = df.groupby('prompt').apply(transform2dpo, weights, include_groups=True)


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/2.86k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/wassname/helpsteer2_dpo_nonverbose/commit/a374f63f0bcf07b6d198f5a6c127099fc13fa80d', commit_message='Upload dataset', commit_description='', oid='a374f63f0bcf07b6d198f5a6c127099fc13fa80d', pr_url=None, pr_revision=None, pr_num=None)

In [9]:
# dataset2.save_to_disk('../data/helpsteer2')

In [8]:
1/0
dataset2.push_to_hub('helpsteer2_dpo_nonverbose', config_name='correctness')

ZeroDivisionError: division by zero